In order to run this notebook, you will first need to run a dask scheduler and number of dask workers:
- Run a dask scheduler with:  ```dask-scheduler --scheduler-file=cluster.json```
- Run N dask workers with:  ```mpirun -np N dask-mpi --no-nanny --nthreads 10 --no-scheduler --scheduler-file cluster.json```

In [1]:
from dask.distributed import Client

In [2]:
# Run this if you are using an MPI-based cluster
client = Client(scheduler_file="cluster.json")

In [3]:
devs = [0, 1]
workers = list(client.has_what().keys())
worker_devs = workers[0:min(len(devs), len(workers))]

In [4]:
def set_visible(i, n):
    import os
    all_devices = list(range(n))
    vd = ",".join(map(str, all_devices[i:] + all_devices[:i]))
    print(str(vd))
    print("Selecting Device : "  + str(i))
    os.environ["CUDA_VISIBLE_DEVICES"] = vd
    
    import numba.cuda
    print("Cur device: " + str(numba.cuda.get_current_device().id))
    
dev_assigned = [client.submit(set_visible, dev, len(devs), workers = [worker]) for dev, worker in zip(devs, worker_devs)]

In [5]:
import cudf
import dask_cudf
import numba.cuda
from dask_cuml.linear_regression import LinearRegression

In [6]:
print(str(dev_assigned))

[<Future: status: pending, key: set_visible-4857c1972e9521babe0b893da0a2d7c4>, <Future: status: pending, key: set_visible-e6121022d7d1e69c7dbff1c6bcbe3152>]


In [7]:
import pandas as pd

X = cudf.DataFrame([('a', [0, 1, 2, 3, 4])])
y = cudf.Series([0, 1, 2, 3, 4])

In [8]:
X_df = dask_cudf.from_cudf(X, chunksize=1).persist()
y_df = dask_cudf.from_cudf(y, chunksize=1).persist()

Set each worker to host dfs on a different device. 

__Note__: You can ignore this if you started your workers with "CUDA_VISIBLE_DEVICE" already

In [9]:
lr = LinearRegression()

In [10]:
res = lr.fit(X_df, y_df)

res

In [12]:
lr.intercept_.result()

5

In [13]:
g = lr.predict(X_df)

WORKER PARTS: [(('10.2.166.167', 34642), <Future: status: finished, type: DataFrame, key: ('from_cudf-fb4e2715580642c0aaf76d5ff6abf23e', 0)>), (('10.2.166.167', 34642), <Future: status: finished, type: DataFrame, key: ('from_cudf-fb4e2715580642c0aaf76d5ff6abf23e', 2)>), (('10.2.166.167', 39157), <Future: status: finished, type: DataFrame, key: ('from_cudf-fb4e2715580642c0aaf76d5ff6abf23e', 1)>), (('10.2.166.167', 39157), <Future: status: finished, type: DataFrame, key: ('from_cudf-fb4e2715580642c0aaf76d5ff6abf23e', 3)>)]
ON WORKER: 2
NOT ON WORKER: 2
IPCHANDLES = [<Future: status: pending, key: get_ipc_handles-30b13741fd4a49da2dd827f64b75f12c>, <Future: status: pending, key: get_ipc_handles-088e70ad6479fa8338d3f13a709d7d36>]
RAW_ARRAYS=[<Future: status: pending, key: as_gpu_matrix-414fb93514c87c433220a2f69fa9a121>, <Future: status: pending, key: as_gpu_matrix-5e393ba1b465a74d19a2de7b3d766887>]


In [14]:
print(str(g.result()))

      
0    1
1    2
2    3
3    4
4    5
